In [1]:
import pandas as pd
import numpy as np
import pygad
import random
import copy

In [2]:
import csv

# Initialize an empty dictionary
enginge_RUL = {}

with open('RUL_consultancy_predictions_A3-2.csv', newline='') as csvfile:
    # Create a CSV reader object with semicolon as delimiter
    reader = csv.reader(csvfile, delimiter=';')
    # Skip the header row
    next(reader)
    # Loop over each row in the CSV
    for row in reader:
        # Check if row has exactly 2 columns
        if len(row) == 2:
            enginge_RUL[int(row[1])] = int(row[0])

In [3]:
engine_RUL_30 = {key: value for key, value in enginge_RUL.items() if value <= 30}

In [4]:
Team_A_time = {key: 4 for key in range(1, 21)}
Team_A_time.update({key: 3 for key in range(21, 56)})
Team_A_time.update({key: 2 for key in range(56, 81)})
Team_A_time.update({key: 8 for key in range(81, 101)})

Team_B_time = {j : Team_A_time[j] + 1 for j in range(1, 26)}
Team_B_time.update({j : Team_A_time[j] + 2 for j in range(26, 71)})
Team_B_time.update({j : Team_A_time[j] + 1 for j in range(71, 101)})

In [5]:
T1_time = Team_A_time
T2_time = Team_B_time
T3_time = Team_A_time
T4_time = Team_B_time

# Random Initializer

In [6]:
def calculate_team_work_time(team, engines):
    if team in ["T1", "T3"]:
        engine_times = Team_A_time
    else:
        engine_times = Team_B_time
    
    time = 0
    
    for engine in engines:
        time += engine_times[engine]
    
    return time
    

def random_init(engines_RUL):
    schedule = {"T1":[], "T2":[], "T3":[], "T4":[]}
    
    possible_engines = list(engines_RUL.keys())
    possible_teams = ["T1", "T2", "T3", "T4"]
    
    iterations = 0 
    
    while possible_engines and iterations < 500:
        index = random.randrange(4)
        team = possible_teams[index]
        
        index = random.randrange(len(possible_engines))
        selected_engine = possible_engines[index]
        
        schedule[team].append(selected_engine)
        
        possible_engines.remove(selected_engine)
        
        iterations += 1

    return schedule

In [7]:
random_init(engine_RUL_30)

{'T1': [20, 24, 40, 34, 77, 31],
 'T2': [36, 91, 61, 76, 37],
 'T3': [41, 90, 100, 56, 42],
 'T4': [35, 92, 49, 68, 82, 81, 66, 64]}

# Loss Function

In [8]:
def engine_fix_date(schedule):
    engine_fix = {}
    for team in schedule:
        days = 0
        if team == "T1" or team == "T3":
            time_table = Team_A_time
        else:
            time_table = Team_B_time
            
        for engine in schedule[team]:
            engine_fix[engine] = days
            days = days + time_table[engine]
              
    return engine_fix


initial_schedule = random_init(engine_RUL_30)
print(initial_schedule)

engine_cost = {key: 4 for key in range(1, 21)}
engine_cost.update({key: 3 for key in range(21, 31)})
engine_cost.update({key: 2 for key in range(31, 46)})
engine_cost.update({key: 5 for key in range(46, 81)})
engine_cost.update({key: 6 for key in range(81, 100)})


def loss_function(schedule, engine_RUL, engine_cost):
    loss = 0
    fix_dates = engine_fix_date(schedule)
    print(fix_dates)
    print(engine_RUL)
    for team in schedule:
        for engine in schedule[team]:

            days_late = fix_dates[engine] - engine_RUL[engine]
            engine_loss = 0
            if days_late > 0:
                cost = engine_cost[engine]
    
                for i in range(days_late):
                    day_loss = cost * (i ** 2)
                    if day_loss > 250:
                        day_loss = 250
                    engine_loss += day_loss
            loss += engine_loss
        
    return loss  

loss_schedule = loss_function(initial_schedule, engine_RUL_30, engine_cost)
print(loss_schedule)

{'T1': [100], 'T2': [82, 40, 91, 35, 90, 81, 34, 37, 41, 24, 49], 'T3': [92, 31, 36, 68, 20, 76, 42], 'T4': [56, 64, 66, 77, 61]}
{100: 0, 82: 0, 40: 9, 91: 14, 35: 23, 90: 28, 81: 37, 34: 46, 37: 51, 41: 56, 24: 61, 49: 65, 92: 0, 31: 8, 36: 11, 68: 14, 20: 16, 76: 20, 42: 22, 56: 0, 64: 4, 66: 8, 77: 12, 61: 15}
{20: 10, 24: 26, 31: 14, 34: 8, 35: 8, 36: 24, 37: 21, 40: 29, 41: 23, 42: 13, 49: 14, 56: 18, 61: 23, 64: 24, 66: 18, 68: 13, 76: 3, 77: 27, 81: 6, 82: 11, 90: 28, 91: 29, 92: 24, 100: 24}
49727


# Termination Criterion

In [9]:
# TO DO LATER

# Selection of parents

In [10]:
# TO DO SELECT ONE DROM PYGAD UTILS
# https://pygad.readthedocs.io/en/latest/pygad.html#parent-selection-methods

# Crossover

In [85]:
parent_1 = random_init(engine_RUL_30)
parent_2 = random_init(engine_RUL_30)

In [143]:
def double_fixer(child, parent):
    doubles = []
    skipped_ones = []
    numbers = {}
    
    for team in parent:
        for engine in parent[team]:
            numbers[engine] = [0]
    
    for team in child:
        index = -1
        for engine in child[team]:
            index = index + 1
            numbers[engine][0] = numbers[engine][0] + 1
            numbers[engine].append(team)
            numbers[engine].append(index)
                
    for key in numbers:
        if numbers[key][0] == 0:
            skipped_ones.append(key)
        elif numbers[key][0] > 1:
            doubles.append(key)
    
    for fault in doubles:
        team = numbers[fault][-2]
        index = numbers[fault][-1]        
        next_missed_one = skipped_ones.pop()
        child[team][index] = next_missed_one
        
    return child

In [144]:
def crossover(parents, offspring_size = 2):
    parent_1 = parents[0]
    parent_2 = parents[1]
    
    child1 = {key: [] for key in parent_1}
    child2 = {key: [] for key in parent_2} 
    cross_side = random.choice(["Left", "Right"]) # Right means you keep the right side of p1 and switch the right side of p2
    print(cross_side)
    for key in parent_1:
        p1_middle = len(parent_1) // 2
        p2_middle = len(parent_2) // 2
        
        if cross_side == "Left":
            sub_schedule_p1 = parent_1[key][:p1_middle]
            sub_schedule_p2 = parent_2[key][:p2_middle]
            
            child1[key].extend(sub_schedule_p2)
            child1[key].extend(parent_1[key][p1_middle:])
            
            child2[key].extend(sub_schedule_p1)
            child2[key].extend(parent_2[key][p2_middle:])
        
        if cross_side == "Right":
            sub_schedule_p1 = parent_1[key][p1_middle:]
            sub_schedule_p2 = parent_2[key][p2_middle:]
            
            child1[key].extend(parent_1[key][:p1_middle])
            child1[key].extend(sub_schedule_p2)
            
            child2[key].extend(parent_2[key][:p2_middle])
            child2[key].extend(sub_schedule_p1)
    
    
    child1 = double_fixer(child1, parent_1)
    child2 = double_fixer(child2, parent_2)
    
    
    return child1, child2
        
    
    

In [152]:
print(parent_1)
print(parent_2)

crossover(parents = [parent_1, parent_2])

{'T1': [40, 34, 82, 35, 81, 100], 'T2': [37, 76, 92], 'T3': [91, 31, 20, 61, 41], 'T4': [64, 66, 42, 68, 90, 56, 49, 24, 36, 77]}
{'T1': [77, 37, 82, 81, 64, 35], 'T2': [76, 49, 61, 92, 66, 40, 100, 31], 'T3': [36, 34, 56, 68, 20], 'T4': [41, 90, 24, 42, 91]}
Left


({'T1': [77, 37, 82, 35, 81, 100],
  'T2': [76, 49, 92],
  'T3': [36, 34, 20, 61, 41],
  'T4': [66, 90, 42, 68, 64, 56, 31, 24, 91, 40]},
 {'T1': [40, 34, 82, 81, 64, 35],
  'T2': [37, 76, 61, 92, 66, 36, 100, 31],
  'T3': [91, 49, 56, 68, 20],
  'T4': [90, 41, 24, 42, 77]})

In [81]:
test = {}
test.update({1 : 0})
test.update({2 : 0})
print(test[1]) 
test[1] = 4
test[1] = test[1] + 1
test[5]
print(test)

0


KeyError: 5

In [82]:
parent_1 = random_init(engine_RUL_30)

child1 = {key: [] for key in parent_1}
child1['T1'].extend([1,2,3])
child1['T1'].extend([4,5,6])
child1

{'T1': [1, 2, 3, 4, 5, 6], 'T2': [], 'T3': [], 'T4': []}

{'T1': [56, 37, 64, 81], 'T2': [91, 41, 40, 68, 100, 92, 36], 'T3': [77, 35, 66, 42, 20], 'T4': [31, 34, 76, 90, 82, 61, 24, 49]}
{'T1': [56, 42, 20, 100], 'T2': [61, 31, 66, 82, 64, 92], 'T3': [36, 77, 49, 34, 91, 24, 76], 'T4': [68, 81, 35, 90, 40, 37, 41]}
Right
{56: 0, 37: 0, 64: 0, 81: 0, 91: 0, 41: 0, 40: 0, 68: 0, 100: 0, 92: 0, 36: 0, 77: 0, 35: 0, 66: 0, 42: 0, 20: 0, 31: 0, 34: 0, 76: 0, 90: 0, 82: 0, 61: 0, 24: 0, 49: 0}


({'T1': [56, 37, 20, 100],
  'T2': [91, 41, 66, 82, 64, 92],
  'T3': [77, 35, 49, 34, 91, 24, 76],
  'T4': [31, 34, 35, 90, 40, 37, 41]},
 {'T1': [56, 42, 64, 81],
  'T2': [61, 31, 40, 68, 100, 92, 36],
  'T3': [36, 77, 66, 42, 20],
  'T4': [68, 81, 76, 90, 82, 61, 24, 49]})